In [1]:
# numpy
import numpy as np
from keras.models import load_model
from sklearn.utils import class_weight
from keras.layers import *
from keras.models import Model
from keras.callbacks import ProgbarLogger, Callback

import json
import tqdm
import os
import pickle
import itertools
import pandas as pd

Using TensorFlow backend.


In [ ]:
callbacks = [EarlyStopping(monitor='val_acc',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_acc',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_acc',
                             filepath='squeeze_net_no_aug.h5',
                             save_best_only=True,
                             save_weights_only=False,
                             mode='max'),
             # TensorBoard(log_dir='logs'),
             CSVLogger('last_training_log.csv', separator=',', append=False)
            ]

# model.load_weights('squeeze_net_no_aug.h5')

model.fit_generator(
            generator=custom_train,
            validation_data = custom_valid,    
            # steps_per_epoch = 10,
            steps_per_epoch=int(np.floor(train_image_count/batch_size)),
            validation_steps =  int(np.floor(valid_image_count/batch_size)),
            epochs=50,
            callbacks=callbacks
        )

## TfIdfVectorizer

In [79]:
class CorporaClass:
    """Class for setting up corpora"""
    def __init__(self):
        self.corpora = []
        self.vocab = set()

    tokenizer = RegexpTokenizer('\w+')
    morph = pymorphy2.MorphAnalyzer()
    ru_pattern = re.compile("[а-яА-Я]")

    @staticmethod
    def full_process(text, tokenizer=tokenizer, morph=morph, ru_pattern=ru_pattern):
        # Clear text from punctuation etc.'''
        tokens = tokenizer.tokenize(text)

        # Turn tokens into normal form excluding non-nouns or verbs
        processed = []
        for token in tokens:
            morphed = morph.parse(token)[0].normal_form
            nf_tag = str(morph.parse(morphed)[0].tag.POS)
            if nf_tag in ("NOUN", "ADJF", "INFN", "NUMR") and len(token) < 16:
                if len(morphed) == len(re.findall(ru_pattern, morphed)):
                    processed.append(morphed)

        result = " ".join(processed)
        return result

    def add_to_corpora(self, file_object):
        try:
            doc = []
            for line in file_object:
                try:
                    processed = self.full_process(line)
                except Exception as e:
                    print(e)
                    processed = ""
                if len(processed):
                    doc.append(processed)
            self.corpora.append(doc)
        except Exception:
            pass
    
    def process_corpora(self):
        all_words = []
        for doc in tqdm.tqdm(self.corpora):
            all_words.extend(list(itertools.chain(*(a.split() for a in doc))))
        vc = pd.Series(all_words).value_counts()
        stoplist = vc.index[:20].tolist() + vc.index[vc.values == 1].tolist()
        new_corpora = []
        for doc in self.corpora:
            accepted_lines = []
            for line in doc:
                accepted_words = []
                for word in line.split():
                    if word not in stoplist:
                        accepted_words.append(word)
                        self.vocab.add(word)
                accepted_lines.append(" ".join(accepted_words))
            new_corpora.append(accepted_lines)
        self.corpora = new_corpora
        self.vocab = self.vocab - {""}

In [ ]:
accepted_docs = ["sport", "elaboration", "military", "medicine", "politics", "safety", "art", "investitions", "finances"]

In [ ]:
corpora_class = CorporaClass()

folders = ["corpora_train", "corpora_test"]
for folder_name in folders:
    for filename in tqdm.tqdm(os.listdir(folder_name)):
        with open(f"{folder_name}/{filename}") as f:
            corpora_class.add_to_corpora(f)
corpora_class.process_corpora()

json.dump(list(corpora_class.vocab), open("vocab.json", "w"))
pickle.dump(corpora_class.corpora, open("corpora.p", "wb"))

 64%|██████▎   | 14/22 [03:28<02:22, 17.85s/it]

no such name
no such name
no such name
no such name
no such name
no such name


 27%|██▋       | 6/22 [01:42<04:05, 15.34s/it]

no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name


 36%|███▋      | 8/22 [04:51<11:32, 49.46s/it]

no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name
no such name


100%|██████████| 44/44 [00:08<00:00,  5.18it/s]


In [24]:
corpora = pickle.load(open("corpora.p", "rb"))
vocab = json.load(open("vocab.json"))

In [53]:
vectorizer = TfidfVectorizer(vocabulary=vocab)
vectors = []
for texts in tqdm.tqdm(corpora):
    try:
        vector = vectorizer.fit_transform(texts)
    except:
        vector = []
    vectors.append(vector)
    
pickle.dump(vectors, open("vectors.p", "wb"))
pickle.dump(vectorizer, open("vectorizer.p", "wb"))

100%|██████████| 44/44 [00:12<00:00,  4.12it/s]


In [54]:
vectors = pickle.load(open("vectors.p", 'rb'))

In [55]:
X_train = vectors[:22]
X_test = vectors[22:]

In [56]:
test_tags_count = json.load(open("test_tags_count.json"))
train_tags_count = json.load(open("train_tags_count.json"))

In [57]:
DELIM = 5000
NUM_OF_CLASSES = len(test_tags_count)
vector_size = X_train[0].shape[1]

In [58]:
from scipy.sparse import vstack

In [77]:
corpora[5]

['привлечение основный общество ответственность сделка дочерний говорить закон',
 'год правовой среда обсуждаться вопрос снятие называть корпоративный вуаль возможность привлекать имущественный ответственность лицо влиять принятие хозяйственный общество решение ответственность основный общество сделка дочерний возможность поднять корпоративный вуаль',
 'новое норма право институт астрент индемнить',
 'июнь год вступить сила изменение один часть рф внести поправка норма обязательный право тот число вопрос ответственность неисполнение обязательство тот быть ввести институт астрент индемнить проанализировать этот другой новелла рф',
 'госдума рф принять третье чтение закон запрет требовать проверка бизнес сведение иметься госорган',
 'дать поправка вносить изменение закон защита право юридический лицо индивидуальный осуществление муниципальный контроль',
 'минфин рф разъяснить деталь предоставление средство федеральный бюджет публичный акционерный общество',
 'ведомство разъяснить изменен

In [59]:
y_train = []
for i, item in enumerate(X_train):
    if item != []:
        num = min(item.shape[0], DELIM)
        for _ in range(num):
            y = np.zeros(NUM_OF_CLASSES)
            y[i] = 1
            y_train.append(y)
y_train = np.array(y_train)

y_test = []
for i, item in enumerate(X_test):
    if item != []:
        num = min(item.shape[0], DELIM)
        for _ in range(num):
            y = np.zeros(NUM_OF_CLASSES)
            y[i] = 1
            y_test.append(y)
y_test = np.array(y_test)

In [60]:
num = min(X_train[0].shape[0], DELIM)
X_train_temp = X_train[0][:num]
for item in X_train[1:]:
    if item != []:
        num = min(item.shape[0], DELIM)
        X_train_temp = vstack([X_train_temp, item[:num]], dtype='float64')
X_train = X_train_temp

num = min(X_test[0].shape[0], DELIM)
X_test_temp = X_test[0][:num]
for item in X_test[1:]:
    if item != []:
        num = min(item.shape[0], DELIM)
        X_test_temp = vstack([X_test_temp, item[:num]], dtype='float64')
X_test = X_test_temp

In [61]:
input_vec = Input(shape=(vector_size,))
l1 = Dense(96, activation='relu')(input_vec)
l2 = Dense(18, activation='relu')(l1)
l3 = Dropout(0.15)(l2)
l4 = Dense(64, activation='sigmoid')(l3)
output = Dense(NUM_OF_CLASSES, activation='softmax')(l4)

In [62]:
classifier = Model(input_vec, output)
classifier.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [63]:
from scipy.sparse import csr_matrix, coo_matrix

In [64]:
X = vstack([X_train, X_test])
y = np.concatenate((y_train, y_test), axis=0)

y = csr_matrix(y)
X = csr_matrix(X, dtype='float32')

In [65]:
X_ = X.toarray()
y_ = y.toarray()

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.3)

In [68]:
classifier.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=196, epochs=250, class_weight='balanced')

Train on 129857 samples, validate on 55653 samples
Epoch 1/250
129857/129857 [==============================] - 64s - loss: 3.0712 - categorical_accuracy: 0.0614 - val_loss: 3.0639 - val_categorical_accuracy: 0.0863
Epoch 2/250
129857/129857 [==============================] - 63s - loss: 2.9892 - categorical_accuracy: 0.1175 - val_loss: 2.8321 - val_categorical_accuracy: 0.1561
Epoch 3/250
129857/129857 [==============================] - 65s - loss: 2.7111 - categorical_accuracy: 0.1775 - val_loss: 2.6238 - val_categorical_accuracy: 0.1979
Epoch 4/250
129857/129857 [==============================] - 66s - loss: 2.5714 - categorical_accuracy: 0.2167 - val_loss: 2.5301 - val_categorical_accuracy: 0.2368
Epoch 5/250
129857/129857 [==============================] - 67s - loss: 2.4810 - categorical_accuracy: 0.2541 - val_loss: 2.4516 - val_categorical_accuracy: 0.2737
Epoch 6/250
129857/129857 [==============================] - 59s - loss: 2.4003 - categorical_accuracy: 0.2871 - val_loss: 2

KeyboardInterrupt: 

['эдинбургский университет обнаружить западный часть антарктида вулкан который расположить ледяной щит сообщать газета',
 'прошлый неделя цена зерно центральный часть россия поволжье обвалиться рубль тонна самый сильный падение сезон рынок давить большой урожай рекордный запас недостаток вагон элеваторный',
 'бывший директор коммуникация белый дом энтони скарамучча интервью заявить президент сша дональд трамп настроить улучшение отношение президент россия владимир путин критика сми господин скарамучча выразить уверенность ситуация быть развиваться положительный ключ сам президент действовать интерес народ сша',
 'воскресение лондон завершиться чемпионат мир атлетика российский спортсмен выступать турнир нейтральный флаг завоевать заключительный день соревнование два серебряный медаль',
 'цб подготовить дорожный карта разработка правило работа площадка оказывать услуга организация взаимный финансирование краудфандинг сектор кредитование гражданин друг друг',
 'девелоперскай группа гране

In [69]:
classifier.save('vk_texts_classifier.h5')

In [70]:
categories = np.array(json.load(open("categories.json")))

In [72]:
categories[classifier.predict(X_[12000].reshape(-1, 10116)).argsort()][0][-3:][::-1].tolist()

['STRATEG', 'BUILDING', 'FINANCES']

In [73]:
categories[y_[12000].argmax()]

'STRATEG'